In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import json
from datetime import datetime
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import openpyxl

In [85]:
def main() : 
    # 나이, 성별 설정 
    # 함수를 이용해서 각 정보를 받아옴
    
    drvier = webdriver.Chrome('C:/Users/concat/Desktop/LinaCrawling/CONCAT/chromedriver_win32/chromedriver.exe')
    url = 'https://www.directdb.co.kr/product/ltm/custInfoLtm.do?searchPdcCd=30581&searchPdcTrtHistCd=00&pdcDvcd=l_tooth' 
    driver.get(url)
    
    wait = WebDriverWait(driver,20)
    
    now_year = datetime.now().year
    
    contents = list()
    
    # 자녀 입력 
    kid_young = int(str((year - 2))  + '1231')
    kid_old   = int(str((year - 19)) + '1231')
    
    for age in range(kid_old, kid_young, 50000) : 
        
        for sxCd in range(2) : 
            
            inputChild(age,sxCd)
            
            years = ['07','15','20']
            
            for year in years : 
                contents += getContent(year,0,sxCd)
            
            

In [86]:
# 자녀형 고객 정보 입력 함수
def inputChild(age, sxCd) : 
    
    global driver
    global wait
    
    driver.find_element_by_class_name("li01").click()
    
    driver.find_element_by_id('birthday').send_keys(age)
    
    sexes = driver.find_element_by_class_name('ico_sex').find_elements_by_class_name('input_radio')
    
    sexes[sxCd-1].find_element_by_tag_name('span').click()
    
    jobkids = driver.find_element_by_class_name('label_horizental').find_elements_by_tag_name('li')
    jobkids[0].find_element_by_tag_name('span').click()

    driver.find_element_by_class_name('btn_foot').click()
    wait.until(EC.invisibility_of_element((By.CLASS_NAME,'loading')))
    


In [87]:
# 성인형 고객 정보 입력 함수

def inputAdult(age, sxCd) : 
    
    global driver
    global wait
    
    driver.find_element_by_id('birthday').send_keys(age)
    
    sexes = driver.find_element_by_class_name('ico_sex').find_elements_by_class_name('input_radio')
    sexes[sxCd-1].find_element_by_tag_name('span').click()
    
    driver.find_element_by_class_name('btn_foot').click()
    wait.until(EC.invisibility_of_element((By.CLASS_NAME,'loading')))

In [82]:
# request를 통해 보장 내역 및 보험료 크롤링

def getContent(year,typecode,sxCd) :
    
    # getLtmPlan.do 
    
    CMKSESSIONID= driver.get_cookie('CMKSESSIONID')['value']
    
    _csrf = driver.find_element_by_name('_csrf').get_attribute('value')
    
    page = driver.page_source
    
    soup = BeautifulSoup(page,'html.parser')
    
    # cvrList 얻기 (보장 내역 코드 리스트)
    cvrlist = soup.find(class_ = 'plan_select').find(class_ = 'plan_name').find_all('dd')
    
    cvr_ = str()
    
    for cvr_e in cvrlist : 
        cvr_t = cvr_e.find(class_='ico_pop').get('name')
        cvr_t = cvr_e.replace('cvrPop_','')
        
        if cvr_ != "" :
            cvr_ += "%2C"
        
        cvr_ += cvr_t
    
    # SlPanCd 값에 따라 유형이 바뀐다. 
    SlPanCd = int(soup.find(id = "searchSlPanCd").get('value'))
    
    plan_slpanCd = [10,0,-10]
    
    contents = list()
    
    # 유형에 따라 반복
    for sl in plan_slpanCd : 
        
        SlPanCd += sl
        
        calc_data = ("searchPdcCd=30581&searchSlPanCd=" + SlPanCd + "&searchPymMtdCd=01" +
            "&searchCvrCds=" + cvr_ + "&searchCalcType=default&pdcDvcd=l_tooth" +
            "&arcTrmCd=Y0"+year+"&pymTrmCd=Y0"+year+"&pymMtdCd=01&_csrf=" + _csrf)
    
        calc_header = {
            'Content-Type' : 'application/x-www-form-urlencoded; charset=UTF-8',
            'X-Requested-With' : 'XMLHttpRequest',
            'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',
            'Referer' : 'https://www.directdb.co.kr/product/ltmSimple/getLtmPlan.do'
        }
        
        calc_cookie = {
             'CMKSESSIONID' : CMKSESSIONID
        }
        
        url = 'https://www.directdb.co.kr/product/ltm/ajaxLtmCalc.do'
        
        response = requests.post(url, headers = calc_header, cookies = calc_cookie, data = calc_data)
        
        # 읽어온 데이터를 json으로 파싱
        data = json.loads(response)
        
        # 내가 입력한 year과 읽어온 데이터의 year이 다르면 경고문 출력
        if year != data['arcTrmCd'] : 
            print('waring! \n year : ',year , '\n data["arcTrmCd"] : ', data['arcTrmCd'] )
        
        
        plan = data['cvrList']
        
        content = dict()

        content['생년월일'] = age
        content['성별'] = '남자' if sxCd == 1 else '여자'
        content['보험기간'] = year

        content['유형'] = data['pdcPanNm']

        for element in plan : 

            name = element['cvrNm']
            inam = element['cvrInam']
            eta = element['etaCn']
            prm = element['cvrPrm']

            if name != None :
                content[name]  = inam

            if eta != None : 
                eta_prices = re.findall(r':([0-9]+)',eta)
                eta_names  = re.findall(r'-([^0-9]+):',eta)

                for i in range(len(eta_names)) : 
                    content[eta_names[i].strip()] = eta_prices[i]
        
        content['보험료'] = data['fstiPrm']

        contents.append(content)
    
    
    return contents

In [16]:
# 해당 보험이 실속형, 기본형, 고급형 중 어떤 것인지 판별하는 함수
def getType(SlPanCd,typecode) : 
    
    typeCd = SlPanCd - typecode 
    
    if typeCd % 30 == 10 : 
        return '고급형'
    elif typeCd % 30 == 20 :
        return '기본형'
    else :
        return '실속형'

In [28]:
calc_header = {
            'Content-Type' : 'application/x-www-form-urlencoded; charset=UTF-8',
            'X-Requested-With' : 'XMLHttpRequest',
            'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',
            'Referer' : 'https://www.directdb.co.kr/product/ltmSimple/getLtmPlan.do'
        }
calc_data = 'searchPdcCd=30581&searchArcAg=44&searchIntAg=44&searchSlPanCd=30&searchPymMtdCd=01&searchPdcTrtHistCd=01&searchCvrCds=307677%2C305065%2C305069%2C305031%2C305093%2C305100%2C307682%2C305379%2C307662&searchPlanAgeCd=&dupCvrCds=&dupPrceCvrCds=&searchCalcType=default&reCalcYn=&atArcEnyYn=&dcInfoCfmYn=&planNo=&pdcDvcd=l_tooth&cmPynTgtClsfCd=&chdDvcd=&nonStopYn=&arcTrmCd=Y007&pymTrmCd=Y007&pymMtdCd=01&_csrf=8937aae0-cb2c-4470-9c96-cb06cfbf3dfb'

cookie = {
    'CMKSESSIONID' : '0001dYOwO0bxits-GJf2LBDDYm-:1aafrjfnb'
}


test = requests.post('https://www.directdb.co.kr/product/ltm/ajaxLtmCalc.do',headers = calc_header, cookies = cookie, data = calc_data)

In [29]:
test_j = json.loads(test.text)

In [32]:
plans = test_j['planCvrLists']

In [63]:
import re 

eta_prices = re.findall(r':([0-9]+)',contents[2]['eta'])
eta_names  = re.findall(r'-([^0-9]+):',contents[2]['eta'])

In [55]:
contents[2]['eta']

'- 임플란트 치료:1500000<br/>- 브릿지 치료:750000<br/>- 틀니 치료:1500000'

In [80]:
plan = test_j['cvrList']
        
content = dict()

# content['생년월일'] = age
# content['성별'] = '남자' if sxCd == 1 else '여자'
# content['보험기간'] = year

content['유형'] = test_j['pdcPanNm']

for element in plan : 

    name = element['cvrNm']
    inam = element['cvrInam']
    eta = element['etaCn']
    prm = element['cvrPrm']

    if name != None :
        content[name]  = inam

    if eta != None : 
        eta_prices = re.findall(r':([0-9]+)',eta)
        eta_names  = re.findall(r'-([^0-9]+):',eta)

        for i in range(len(eta_names)) : 
            content[eta_names[i].strip()] = eta_prices[i]

content['보험료'] = test_j['fstiPrm']

In [81]:
content

{'유형': '실속형',
 '치아보존치료비(상해및질병)': '200000',
 '아말감 또는 글래스아이노머(G.I)': '20000',
 '레진필링, 인레이, 온레이 등': '100000',
 '크라운 치료': '200000',
 '영구치보철치료비(상해및질병)': '500000',
 '임플란트 치료': '500000',
 '브릿지 치료': '250000',
 '틀니 치료': '500000',
 '영구치상실치료비(상해및질병)': '30000',
 '치주질환치료비': '30000',
 '치아치수치료비(상해및질병)': '20000',
 '보험료': '25490'}